<a href="https://colab.research.google.com/github/harjeet88/A_For_Algorithms/blob/master/crew_ai/yt_shorts_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install required pacakges


In [1]:
!pip install -q -U crewai
!pip install -q -U duckduckgo-search
!pip install -q -U crewai_tools

### Required Packages

In [3]:
import os

from crewai import Agent, Task, Crew, Process, LLM
#from crewai_tools import tool

In [9]:
from crewai.tools import BaseTool

In [4]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
os.environ["GEMINI_API_KEY"] = GOOGLE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
if GOOGLE_API_KEY :
  print("GOOGLE_API_KEY is set")

GOOGLE_API_KEY is set


### set up the LLM

In [48]:
from crewai import Agent, LLM

gemini = LLM(
    api_key=os.getenv("GEMINI_API_KEY"),
    model="gemini/gemini-2.5-flash",
)

In [49]:
my_llm=gemini

### setup tools

In [50]:
!pip install --upgrade --quiet  langchain-community

In [66]:
story_teller = Agent(
    role = 'pro content creator for youtube shorts',
    goal = '''Take the high level topic described here and prepare a very intresting and very engrosing story for a youtube video.
    Story should be narrated within four minutes. strory should have intresting start and amzing end and should ensure listener is hooked throught story.
    The python module must be completely self-contained, and ready so that it can be tested or have a simple UI built for it.
    here is the topic: {topic}''',
    backstory = '''You're a seasoned a seasoned story teller reknowned for compelling story telling''',
    llm = my_llm,
    verbose=True,
)

In [68]:
photographer = Agent(
    role = 'you create very compelling photo with great detail',
    goal = ''' you create photos based on story told by story teller and topic. you take for every 2 sentences in story, you create one photo which describe the scene in great detail.''',
    backstory = '''You're a seasoned photo creater with a knack for creating amazingly attractive photos''',
    llm = my_llm,
    context=[story_teller],
    verbose=True,
)

### Tasks to perform

In [69]:
# Create tasks for your agents
story_teller_task = Task(
  description="""Take the given topic {topic} and write a captivating YouTube video story that can be narrated in four minutes. The story must have an intriguing start, an engaging middle, and an amazing end. Structure the story into three labeled parts: 'Beginning', 'Middle', and 'End'. Also, provide a catchy title at the top. The entire output must be in plain text without any markdown""",
  agent=backend_engineer,
  verbose=True,
  expected_output="""Expected Output:

A string in the following format:

"Title: [Title of the story]

Beginning: [Text for the beginning]

Middle: [Text for the middle]

End: [Text for the end]"

The story should be approximately 500-600 words in total."""

)

In [56]:
photographer_task = Task(
  description="""""",
  agent=backend_engineer,
  expected_output="""A python module that implements the design and achieves the requirements.
    IMPORTANT: Output ONLY the raw Python code without any markdown formatting, code block delimiters, or backticks.
    The output should be valid Python code that can be directly saved to a file and executed.""",
  context=[design_task],
  output_file= "output/{module_name}"
)

In [57]:
frontend_task = Task(
  description="""Write a gradio UI in a module app.py that demonstrates the given backend class in {module_name}.
    Assume there is only 1 user, and keep the UI very simple indeed - just a prototype or demo.
    Here are the requirements: {requirements}""",
  agent=frontend_engineer,
  expected_output="""A gradio UI in module app.py that demonstrates the given backend class.
    The file should be ready so that it can be run as-is, in the same directory as the backend module, and it should import the backend class from {module_name}.
    IMPORTANT: Output ONLY the raw Python code without any markdown formatting, code block delimiters, or backticks.
    The output should be valid Python code that can be directly saved to a file and executed.""",
  context=[code_task],
  output_file= "output/app.py"
)

In [58]:
test_task = Task(
  description="""Write unit tests for the given backend module {module_name} and create a test_{module_name} in the same directory as the backend module.""",
  agent=test_engineer,
  expected_output="""A test_{module_name} module that tests the given backend module.
    IMPORTANT: Output ONLY the raw Python code without any markdown formatting, code block delimiters, or backticks.
    The output should be valid Python code that can be directly saved to a file and executed.""",
  context=[code_task],
  output_file= "output/test_{module_name}"
)

### Create a Crew

In [70]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[story_teller],
  tasks=[story_teller_task],
  verbose=False, # You can set it to 1 or 2 to different logging levels
)

In [71]:
crew

Crew(id=a2f90f7f-a2a1-4f12-9fcd-fb4d21ab835f, process=Process.sequential, number_of_agents=1, number_of_tasks=1)

### Kickoff the crew - let the magic happen

In [72]:
topic = """ indian historical stories
"""

template_input = {"topic": topic}



In [73]:
# Get your crew to work!
#result = crew.kickoff()
result = crew.kickoff(inputs=template_input)

# Agent: Python Engineer who can write code to achieve the design described by the engineering lead
## Task: Take the given topic  indian historical stories
 and write a captivating YouTube video story that can be narrated in four minutes. The story must have an intriguing start, an engaging middle, and an amazing end. Structure the story into three labeled parts: 'Beginning', 'Middle', and 'End'. Also, provide a catchy title at the top. The entire output must be in plain text without any markdown


# Agent: Python Engineer who can write code to achieve the design described by the engineering lead
## Final Answer: 
Title: The Emperor's Conversion: Ashoka and the Tide of Blood

Beginning:
In the heart of ancient India, a shadow stretched across the land, cast by the ambition of Emperor Ashoka. He was a force of nature, a whirlwind of conquest, his empire already vast, yet his hunger for dominion remained unquenched. Stories of his ferocity, his merciless drive, echoed through the kingdo

In [74]:
print(result)

Title: The Emperor's Conversion: Ashoka and the Tide of Blood

Beginning:
In the heart of ancient India, a shadow stretched across the land, cast by the ambition of Emperor Ashoka. He was a force of nature, a whirlwind of conquest, his empire already vast, yet his hunger for dominion remained unquenched. Stories of his ferocity, his merciless drive, echoed through the kingdom of Magadha. He was Chandashoka, Ashoka the Fierce, and his gaze was fixed upon Kalinga, a prosperous, independent kingdom resisting his might. The year was 261 BCE. Ashoka desired Kalinga not just for its strategic location, but as a testament to his absolute power, a final jewel in his imperial crown. His armies, a crushing wave of human might, numbering hundreds of thousands, marched towards Kalinga, promising a conflict that would etch itself into the annals of history, a clash so cataclysmic it would change the course of an empire, and indeed, an emperor's soul, forever. The air itself seemed to hum with the t